<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/claude4_aiagent_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_anthropic -q
!pip install crewai -q
!pip install colab-env -q
!pip install anthropic -q
import colab_env

In [11]:
import os
import anthropic
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from langchain_anthropic import ChatAnthropic
from pydantic import BaseModel, Field

# --- Configuration: Retrieve API Key ---
# Assuming colab_env has already been run in a previous cell
api_key = os.environ.get("CLAUDE3_API_KEY")

if not api_key:
    raise ValueError("CLAUDE3_API_KEY environment variable not set. "
                     "Please set it (e.g., export CLAUDE3_API_KEY='sk-ant-...')")

# Define the LLMs for each agent using langchain_anthropic

opus_llm = ChatAnthropic(model="claude-opus-4-20250514", temperature=0.7, anthropic_api_key=api_key)
sonnet_llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0.5, anthropic_api_key=api_key)

# --- Custom Tools (None for this easy task, but keeping the class definitions for structure) ---
# We won't assign these to the agent for this specific easy task,
# to ensure no tool-related recursion issues.

class WeatherQueryInput(BaseModel):
    query: str = Field(description="A specific flight identifier for which to fetch detailed weather.")
class NOTAMQueryInput(BaseModel):
    query: str = Field(description="A specific airport ICAO code or a route identifier for which to fetch NOTAMs.")
class FlightInfoInput(BaseModel):
    flight_info: str = Field(description="A string containing essential flight details for a quick weather summary.")

class DetailedWeatherTool(BaseTool):
    name: str = "Detailed Weather Fetcher"
    description: str = "Fetches comprehensive, granular weather data."
    args_schema: BaseModel = WeatherQueryInput
    def _run(self, query: str) -> str: return "Simulated weather data."

class NOTAMFetcherTool(BaseTool):
    name: str = "NOTAM Fetcher"
    description: str = "Retrieves active Notices to Airmen (NOTAMs)."
    args_schema: BaseModel = NOTAMQueryInput
    def _run(self, query: str) -> str: return "Simulated NOTAM data."

class QuickWeatherSummaryTool(BaseTool):
    name: str = "Quick Weather Summary"
    description: str = "Provides a very brief, high-level summary of critical weather conditions."
    args_schema: BaseModel = FlightInfoInput
    def _run(self, flight_info: str) -> str: return "Simulated quick weather data."

# Instantiate tools (not assigned to agent for this easy task, just defined)
detailed_weather_tool = DetailedWeatherTool()
notam_fetcher_tool = NOTAMFetcherTool()
quick_weather_tool = QuickWeatherSummaryTool()

# --- Define Agents ---

# Risk Assessment Agent (Still defined, but not used in this easy task)
risk_assessment_agent = Agent(
    role="Aviation Safety Analyst",
    goal="Perform comprehensive risk assessments for complex flight scenarios.",
    backstory="You are a highly experienced and meticulous aviation safety analyst.",
    llm=opus_llm,
    tools=[], # No tools for this simple task
    verbose=False, # Changed to False
    allow_delegation=False
)

# Flight Briefing Agent (Sonnet-powered) - This is the agent we'll use
flight_briefing_agent = Agent(
    role="Pilot Briefing Specialist",
    goal="Deliver concise, immediate, and critical aviation information.",
    backstory="You are a fast and efficient briefing specialist, known for providing interesting aviation facts.",
    llm=sonnet_llm, # Using Claude Sonnet 4
    tools=[], # No tools needed for a fun fact
    verbose=False, # Changed to False
    allow_delegation=False
)

# --- Define Tasks ---

# An easy task for the Flight Briefing Agent
easy_fun_fact_task = Task(
    description=(
        "Tell me a fascinating and lesser-known fun fact about aviation history or technology. "
        "Keep it concise, interesting, and engaging for a general audience."
    ),
    expected_output="A single, fascinating, concise aviation fun fact.",
    agent=flight_briefing_agent, # Assign to the Sonnet-powered agent
)

# --- Create the Crew ---

# Crew for the easy fun fact task
fun_fact_crew = Crew(
    agents=[flight_briefing_agent], # Only the Sonnet-powered agent
    tasks=[easy_fun_fact_task],
    process=Process.sequential,
    verbose=False # Changed to False
)

# --- Run the Crew ---

if __name__ == "__main__":
    print("\n" + "#"*80)
    print("## Initiating Fun Fact Generation ##".center(76))
    print("#"*80 + "\n")
    # Run the crew and get the CrewOutput object
    crew_output = fun_fact_crew.kickoff()
    # The final result is often available in the `final_answer` attribute
    # or by simply printing the CrewOutput object which stringifies to the final answer.
    # Let's be explicit and get the final task output if available.
    if hasattr(crew_output, 'tasks_outputs') and crew_output.tasks_outputs:
        # In sequential process, the last task output is usually the final result
        fun_fact_result = crew_output.tasks_outputs[-1].result
    elif hasattr(crew_output, 'final_answer'):
         # Alternative attribute depending on crewai version/process
         fun_fact_result = crew_output.final_answer
    else:
         # Fallback to the string representation of the output
         fun_fact_result = str(crew_output)

    print("\n" + "#"*80)
    print("## FINAL FUN FACT RESULT ##".center(76))
    print("#"*80)
    # Print the explicitly extracted or fallback result
    print(fun_fact_result)


################################################################################
                    ## Initiating Fun Fact Generation ##                    
################################################################################


################################################################################
                        ## FINAL FUN FACT RESULT ##                         
################################################################################
Here's a fascinating aviation fact: The Wright brothers' first flight on December 17, 1903, lasted only 12 seconds and covered just 120 feet - shorter than the wingspan of a modern Boeing 747! Even more remarkable, the total airtime for all four flights that historic day was less than two minutes combined. Yet this brief moment in the air changed human history forever, proving that powered flight was possible and launching the aviation age that would transform our world within just a few decades.
